In [1]:
!pip install --upgrade --quiet pinecone-client pinecone-text pinecone-notebooks

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 6.9 MB/s eta 0:00:00


In [1]:
api_key = #add your pinecone api key

In [3]:
!pip install langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 60.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 3.7 MB/s eta 0:00:00


In [4]:
!pip install langchain_huggingface

In [2]:
from langchain_community.retrievers import PineconeHybridSearchRetriever

In [3]:
import os
from pinecone import Pinecone,ServerlessSpec
index_name="hybrid-search-langchain-pinecone"
## initialize pinecone client
pc=Pinecone(api_key=api_key)

# create the index
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=384, #dimension of dense vector, depends on embedding model dimension
        metric="dotproduct",  ## sparse values supported only for dotproduct
        spec=ServerlessSpec(cloud='aws', region='us-east-1'),
    )

In [4]:
index=pc.Index(index_name)
index

In [6]:
#vector embedding and sparse matrix
import os
from dotenv import load_dotenv
load_dotenv()
os.environ["HUGGINGFACEHUB_API_TOKEN"]=os.getenv("HUGGINGFACEHUB_API_TOKEN")
from langchain_huggingface import HuggingFaceEmbeddings
embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
embeddings

HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [10]:
##required for tokenization
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [11]:
from nltk.tokenize import word_tokenize

sentences = [
    "In 2023, I visited Paris",
    "In 2022, I visited New York",
    "In 2021, I visited New Orleans",
]

# Tokenize sentences
tokenized_sentences = [word_tokenize(sentence) for sentence in sentences]
print("Tokenized Sentences:", tokenized_sentences)

Tokenized Sentences: [['In', '2023', ',', 'I', 'visited', 'Paris'], ['In', '2022', ',', 'I', 'visited', 'New', 'York'], ['In', '2021', ',', 'I', 'visited', 'New', 'Orleans']]


In [12]:
from pinecone_text.sparse import BM25Encoder #uses tfidf by default

bm25_encoder = BM25Encoder().default()

bm25_encoder #responsible for sparse matrix creation

In [13]:
sentences = [
    "In 2023, I visited Paris",
        "In 2022, I visited New York",
        "In 2021, I visited New Orleans",
]
# tfidf values on these sentences
bm25_encoder.fit(sentences)

# store the values to json file
bm25_encoder.dump("bm25_values.json")

# load to your BM25Encoder object
bm25_encoder = BM25Encoder().load("bm25_values.json")

  0%|          | 0/3 [00:00<?, ?it/s]

In [14]:
##retriver includes both the components, dense vector and sparse vectors as two components to retrieve
retriever=PineconeHybridSearchRetriever(embeddings=embeddings,sparse_encoder=bm25_encoder,index=index)
retriever

PineconeHybridSearchRetriever(embeddings=HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False), sparse_encoder=<pinecone_text.sparse.bm25_encoder.BM25Encoder object at 0x7ccba2d8bf90>, index=<pinecone.data.index.Index object at 0x7cccc2e89c10>)

In [15]:
retriever.add_texts(texts=["In 2023, I visited Paris",
        "In 2022, I visited New York",
        "In 2021, I visited New Orleans",
])

  0%|          | 0/1 [00:00<?, ?it/s]

In [17]:
retriever.invoke("which city did i visit first")

[Document(metadata={'score': 0.239936098}, page_content='In 2021, I visited New Orleans'),
 Document(metadata={'score': 0.232014373}, page_content='In 2022, I visited New York'),
 Document(metadata={'score': 0.220718235}, page_content='In 2023, I visited Paris')]

In [18]:
retriever.invoke("which city did i visit in 2021?")

[Document(metadata={'score': 0.461440027}, page_content='In 2021, I visited New Orleans'),
 Document(metadata={'score': 0.314810485}, page_content='In 2022, I visited New York'),
 Document(metadata={'score': 0.301422924}, page_content='In 2023, I visited Paris')]

In [19]:
retriever.invoke("which city did i visit in 2022?")

[Document(metadata={'score': 0.490224838}, page_content='In 2022, I visited New York'),
 Document(metadata={'score': 0.35527432}, page_content='In 2023, I visited Paris'),
 Document(metadata={'score': 0.339998931}, page_content='In 2021, I visited New Orleans')]

In [20]:
retriever.invoke("when did I visit Paris")

[Document(metadata={'score': 0.634853959}, page_content='In 2023, I visited Paris'),
 Document(metadata={'score': 0.240954757}, page_content='In 2021, I visited New Orleans'),
 Document(metadata={'score': 0.20252192}, page_content='In 2022, I visited New York')]